In [1]:
import json
import pickle
import random
from collections import defaultdict
from datetime import datetime
from itertools import chain
from math import log
from typing import Iterable

import keras
import numpy
import pandas
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from matplotlib import pyplot
from scipy import stats
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, StratifiedKFold
from statsmodels.formula import api as formula_api

%matplotlib inline
pandas.set_option('display.max_columns', 500)

Using TensorFlow backend.


In [2]:
# Define cross-validation and evaluation strategies.

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = 'accuracy'

In [3]:
# Parse battle logs.

def parse_heroes(heroes: Iterable[dict], sign: int, result: dict):
    for hero in heroes:
        result[f'level_{hero["id"]}'] += sign * hero['level']
        result[f'color_{hero["id"]}'] += sign * hero['color']
        result[f'star_{hero["id"]}'] += sign * hero['star']


def parse_battle(line: str) -> dict:
    battle = json.loads(line)
    result = defaultdict(int)

    parse_heroes(battle.get('attackers') or battle['player'], +1, result)
    parse_heroes(battle.get('defenders') or battle['enemies'], -1, result)
        
    return {'win': battle['win'], **result}

In [4]:
# Load the logs into a data frame.

lines = list(chain.from_iterable([
    open('battles.jsonl'),
    open('battles-lilia.jsonl'),
]))

numpy.random.seed(42)
battles = pandas.DataFrame([parse_battle(line) for line in lines])
battles.drop_duplicates(inplace=True)  # because the files may contain duplicate battles
battles.dropna(axis=1, thresh=5, inplace=True)  # require some number of battles for each hero
battles.fillna(value=0.0, inplace=True)
print(f'Shape: {battles.shape}')
battles.head()

Shape: (350, 109)


,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_32,color_33,color_34,color_35,color_36,color_38,color_39,color_4,color_40,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_32,level_33,level_34,level_35,level_36,level_38,level_39,level_4,level_40,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_32,star_33,star_34,star_35,star_36,star_38,star_39,star_4,star_40,star_5,star_6,star_7,star_8,star_9,win
0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,-2.0,-5.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,35.0,0.0,-42.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,-7.0,-42.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-2.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False
1,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,-2.0,-6.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,-10.0,0.0,0.0,35.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,-13.0,-46.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,-3.0,0.0,False
2,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,-2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,36.0,0.0,0.0,-9.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-42.0,0.0,0.0,0.0,-45.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,36.0,0.0,0.0,35.0,0.0,-10.0,-46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-1.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,37.0,0.0,0.0,37.0,0.0,-4.0,0.0,0.0,-41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,-41.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False


In [5]:
battles.describe()

,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_32,color_33,color_34,color_35,color_36,color_38,color_39,color_4,color_40,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_32,level_33,level_34,level_35,level_36,level_38,level_39,level_4,level_40,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_32,star_33,star_34,star_35,star_36,star_38,star_39,star_4,star_40,star_5,star_6,star_7,star_8,star_9
count,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000
mean,-0.485714,3.257143,1.500000,-1.245714,3.602857,-0.374286,-0.817143,0.062857,-0.400000,0.182857,-0.180000,1.565714,-1.420000,-0.174286,-0.180000,1.505714,-0.388571,-0.285714,-0.128571,-0.108571,-0.202857,-0.140000,-0.328571,-0.085714,-0.140000,0.188571,-0.134286,0.302857,-0.154286,-2.477143,-0.088571,-0.102857,-0.791429,-0.480000,-0.880000,-0.325714,-3.082857,27.368571,11.774286,-7.997143,28.165714,-2.791429,-6.237143,0.640000,-3.231429,2.922857,-1.414286,11.542857,-11.260000,-1.374286,-1.448571,11.865714,-3.114286,-2.180000,-0.962857,-0.814286,-1.497143,-1.108571,-2.734286,-0.777143,-1.062857,1.677143,-1.011429,2.568571,-1.257143,-19.334286,-0.834286,-0.828571,-5.162857,-3.502857,-7.202857,-1.882857,-0.154286,1.854286,0.694286,-0.491429,1.622857,-0.140000,-0.217143,0.034286,-0.160000,0.091429,-0.060000,0.825714,-0.491429,-0.068571,-0.057143,0.085714,-0.125714,-0.114286,-0.040000,-0.031429,-0.060000,-0.088571,-0.114286,-0.045714,-0.051429,0.074286,-0.037143,0.134286,-0.045714,-0.962857,-0.002857,-0.051429,-0.231429,0.088571,-0.328571,-0.165714
std,2.755265,3.864042,3.428141,3.618984,3.925450,1.949189,5.133824,0.946441,1.680838,3.900737,1.455805,3.880740,3.183617,1.269398,1.072795,4.255281,1.624626,1.475132,1.503782,0.905055,1.289807,1.060005,1.530225,0.901175,1.273647,2.052013,1.153900,1.742686,1.152633,4.372785,0.866859,0.860294,2.607996,4.841724,2.577709,1.369823,22.343797,31.764510,27.100427,29.490565,30.688489,15.580990,41.831608,8.002979,13.543124,31.837649,11.413174,31.451553,26.144153,9.857723,8.696173,33.850027,12.855760,11.346289,12.733382,6.810772,10.029398,8.757130,12.142219,7.772300,10.138386,17.489458,9.448087,14.697509,9.375768,35.547131,7.105103,6.936824,21.966484,38.525305,21.127705,10.808863,1.015169,2.108496,1.570035,1.720827,1.856003,0.815572,2.176652,0.459190,0.679288,1.543147,0.659382,1.444651,1.227052,0.485770,0.333674,1.364121,0.541854,0

In [6]:
# Split the dataframe.

x = battles.drop(['win'], axis=1)
y = battles['win']

In [7]:
# Train, adjust hyper-parameters and evaluate.

numpy.random.seed(42)

classifier = RandomForestClassifier(class_weight='balanced', n_jobs=5, random_state=42)
param_grid = {
    'n_estimators': list(range(1, 501, 5)),
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None],
    'criterion': ['entropy', 'gini'],
}

%time grid_search = RandomizedSearchCV(classifier, param_grid, cv=cv, scoring=scoring, n_iter=20).fit(x, y)
best_estimator = grid_search.best_estimator_

print(f'Best score: {grid_search.best_score_}')
print(f'Best params: {grid_search.best_params_}')
print(f'Classes: {best_estimator.classes_}')

scores = cross_val_score(best_estimator, x, y, scoring=scoring, cv=cv)
print(f'CV score: {scores.mean()} (std: {scores.std()})')

CPU times: user 55.2 s, sys: 7.09 s, total: 1min 2s
Wall time: 1min 4s
Best score: 0.7914285714285715
Best params: {'n_estimators': 131, 'max_features': 'log2', 'max_depth': None, 'criterion': 'entropy'}
Classes: [False  True]
CV score: 0.7912098679030706 (std: 0.03872225123075208)


In [8]:
# Re-train the best model on the entire dataset.

best_estimator.fit(x, y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=None, max_features='log2',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=131, n_jobs=5,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [9]:
# Save the model.

print(f'''
"""
Arena battle prediction model.
Auto-generated on {datetime.now().replace(microsecond=0)}.
X shape: {x.shape}.
Score: {scores.mean():.4f} (std: {scores.std():.4f}).
"""

import pickle

from {best_estimator.__class__.__module__} import {best_estimator.__class__.__qualname__}

feature_names = {list(x.columns)}
model: {best_estimator.__class__.__qualname__} = pickle.loads({pickle.dumps(best_estimator)})
'''.strip(), file=open('model.py', 'wt'))

In [10]:
# Display the feature importances.

pandas.DataFrame({'Feature': x.columns, 'Importance': best_estimator.feature_importances_}) \
    .set_index('Feature') \
    .sort_values('Importance', ascending=False) \
    .head(n=10)

,Importance
Feature,
level_7,0.044902
level_2,0.044264
level_15,0.038828
level_25,0.031053
level_10,0.030551
color_7,0.030426
level_4,0.029308
level_9,0.028722
level_18,0.025791


In [11]:
# Display prediction examples.

result = pandas.concat((
    pandas.Series(best_estimator.predict(x), index=battles.index, name='win_predicted'),
    pandas.Series(best_estimator.predict_proba(x)[:, 1], index=battles.index, name='win_probability'),
    battles,
), axis=1)
result['win_probability'] = result['win_probability'].apply('{:.2f}'.format)
result.head(10)

,win_predicted,win_probability,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_32,color_33,color_34,color_35,color_36,color_38,color_39,color_4,color_40,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_32,level_33,level_34,level_35,level_36,level_38,level_39,level_4,level_40,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_32,star_33,star_34,star_35,star_36,star_38,star_39,star_4,star_40,star_5,star_6,star_7,star_8,star_9,win
0,False,0.09,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,-2.0,-5.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,35.0,0.0,-42.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,-7.0,-42.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-2.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False
1,False,0.01,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,-2.0,-6.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,-10.0,0.0,0.0,35.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,-13.0,-46.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,-3.0,0.0,False
2,False,0.08,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,-2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,36.0,0.0,0.0,-9.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-42.0,0.0,0.0,0.0,-45.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3,False,0.05,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,36.0,0.0,0.0,35.0,0.0,-10.0,-46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-1.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4,False,0.10,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,37.0,0.0,0.0,37.0,0.0,-4.0,0.0,0.0,-41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,-41.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False
5,False,0.02,0.0,0.0,0.0,-3.0,0.0,0.0,-1.0,0.0,0.0,4.0,0.0,-1.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,0.0,-13.0,0.0,0.0,-13.0,0.0,0.0,37.0,0.0,-9.0,-46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
6,False,0.05,0.0,-5.0,0.0,-1.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,-42.0,0.0,-5.0,0.0,0.0,-5.0,0.0,0.0,37.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0